<a href="https://colab.research.google.com/github/redish0317/cp1/blob/main/cp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
import time
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import warnings
import re
warnings.simplefilter(action='ignore', category=FutureWarning)


def return_id_list(tag_type_list):
    '''
    1번째문제:
    0번째 게시물부터 100개씩 크롤링 while true
    오류발생! => ex) 총 게시물이 321개인데 300개 크롤링 후 다음100개를 크롤링하려했기때문
    따라서 재귀호출을 통해 크롤링 수를 100개씩 -> 오류발생! -> 10개씩 -> 오류발생! -> 1개씩 크롤링하는 함수구현
    만약 게시물이 321개라면 300개 크롤링 -> 20개 크롤링 - 1개 크롤링 return

    2번째문제:
    [1024, 1025, 10231, 1634, 655] # 데이터사이언티스트, 빅데이터엔지니어, DBA, 머신러닝엔지니어, 데이터엔지니어를
    쿼리에 한번에 넣으니 합집합이아닌 교집합만 출력됨
    따라서 return_id_list에 list형태로 ex)[1024, 1025, 10231, 1634, 655] 값을 넣으면 각 값의 id를 모두 추출하고 중복을 제거하는
    방식으로 변경

    코드 사용 예시:
    tag_type_list = [1024, 1025, 10231, 1634, 655] # 데이터사이언티스트, 빅데이터엔지니어, DBA, 머신러닝엔지니어, 데이터엔지니어
    id_list = return_id_list(tag_type_list)
    '''
    def crawl_id(tag,limit=100,offset=0):
        url = 'http://www.wanted.jobs/api/v4/jobs?'

        params ={1656232918453:'',    #사용자번호?
        'country': 'all',
        'tag_type_ids': tag, #개발직
        'job_sort': 'job.latest_order',    #최신순 정렬
        'locations': all,
        'years': -1,    #경력 이상    경력상관없이 검색하려면 -1 , 신입은0
        'years': -1,    #경력 이하    경력상관없이 검색하려면 -1 , 신입은0
        'limit': limit,    #한 번에 조회 가능한 수 (최대100)
        'offset': offset}    #조회할 게시물의 첫 index        ex) limit=100 offset=10  => 10번게시물부터 110번게시물까지 크롤링

        #서버에 url과 쿼리로 요청
        r = requests.get(url,
                        params = params)
        #요청한 데이터 json포멧으로 변환
        r = r.json()
        #json포멧 데이터중 id컬럼만 추출
        id_list = [i['id'] for i in r['data']]
        return id_list

    def crawl_all_id(tag,limit=100,offset=0):
        try:
            while True:
                id_list.extend(crawl_id(tag,limit,offset))
                offset+=limit

        except:
            if limit != 1:
                return crawl_all_id(tag,limit/10,offset)
    
    id_list=[]

    for tag in tag_type_list:
        crawl_all_id(tag)

    return list(set(id_list))#중복제거


def crawl_job(id_list):
    df_list = []
    
    for id in id_list:
        url = f'https://www.wanted.jobs/api/v4/jobs/{id}?1656259528432'
        r = requests.get(url)
        r = r.json()['job']

        #1개의 게시물 크롤링할때마다 데이터프레임에 append 또는 concat하는것보다
        #list에 append하고 마지막에 한번에 concat하는게 속도가 더 빠르다고 함
        df_list.append(pd.json_normalize(r))
        
    df = pd.concat(df_list, ignore_index=True)
    return df


def engineering(df):
    #사용하지 않을 컬럼 drop
    drop_col = ['is_crossboarder','like_count','is_like','score','company_images','status','is_bookmark','is_company_follow',
            'compare_country','matching_score','short_link','address.geo_location.n_location.address','address.geo_location.location.lat','address.geo_location.location.lng','address.geo_location.viewport.northeast.lat',
            'address.geo_location.viewport.northeast.lng','address.geo_location.viewport.southwest.lat','address.geo_location.viewport.southwest.lng','address.geo_location.bounds','detail.intro','detail.benefits','company.id','company.application_response_stats.level',
            'company.application_response_stats.delayed_count', 'company.application_response_stats.remained_count', 'company.application_response_stats.type', 'logo_img.origin', 'logo_img.thumb', 'title_img.origin', 'title_img.thumb', 'reward.formatted_total', 'reward.formatted_recommender', 'reward.formatted_recommendee', 'address.geo_location.bounds.northeast.lat', 'address.geo_location.bounds.northeast.lng', 'address.geo_location.bounds.southwest.lat', 'address.geo_location.bounds.southwest.lng','address.geo_location']
    df = df.drop(drop_col,axis=1)

    #직무별 고유 id dict
    category_tags_dict = {873:'웹 개발자', 872:'서버 개발자', 669:'프론트엔드 개발자',10110:'소프트웨어 엔제니어', 660:'자바 개발자',
                        677:'안드로이드 개발자', 678:'iOS 개발자', 895:'Node.js 개발자', 655:'데이터 엔지니어', 899:'파이썬 개발자',
                        674:'DevOps / 시스템 관리자', 900:'C,C++ 개발자', 665:'시스템,네트워크 관리자', 1634:'머신러닝 엔지니어', 1024:'데이터 사이언티스트',
                        1025:'빅데이터 엔지니어', 676:'QA,테스트 엔지니어', 877:'개발 매니저', 1026:'기술지원', 671:'보안 엔지니어',
                        876:'프로덕트 매니저', 1027:'블록체인 플랫폼 엔지니어', 893:'PHP 개발자', 658:'임베디드 개발자', 939:'웹 퍼블리셔',
                        672:'하드웨어 엔지니어', 10111:'크로스플랫폼 앱 개발자', 661:'.NET 개발자', 896:'영상,음성 엔지니어', 10231:'DBA',
                        898:'그래픽스 엔지니어', 795:'CTO', 10112:'VR 엔지니어', 10230:'ERP전문가', 894:'루비온레일즈 개발자',
                        1022:'BI엔지니어', 793:'CIO'}

    try:
        #skill_tags 비어있는 행 제거
        df = df.loc[df['skill_tags'].apply(lambda x :  x !=[])]
        #skill_tags 문자열에서 키워드만 추출
        df['skill_tags'] = df['skill_tags'].apply(lambda x : [list(i.values())[0] for i  in x])
        #company_tags 문자열에서 키워드만 추출
        df['company_tags'] = df['company_tags'].apply(lambda x : [i['title'] for i  in x])
        #category_tags 직무id만 추출
        df['category_tags'] = df['category_tags'].apply(lambda x : [i['id'] for i  in x])
        

    except:#크롤링시 가끔 dict list가 문자열형태로 존재하는 경우가 발생 ex) '[a,b,c,d]', '{a:1,b:2,c:3}' 이런경우 문자열로접근
        #skill_tags 비어있는 행 제거
        df = df.loc[df['skill_tags'].apply(lambda x :  x !='[]')]
        #skill_tags 문자열에서 키워드만 추출
        df['skill_tags'] = df['skill_tags'].apply(lambda x : [i.split(',')[0][2:-1] for i in x.split(':') if 'id' in i])
        #company_tags 문자열에서 키워드만 추출
        df['company_tags'] = df['company_tags'].apply(lambda x : [i.split(',')[0][2:-1] for i in x.split(':') if 'id' in i][:-1])
        #category_tags 직무id만 추출
        df['category_tags'] = df['category_tags'].apply(lambda x : [re.sub(r'[^0-9]', '', v) for i,v in enumerate(x.split(':')) if i % 2 == 0][1:])


    #고유 id로 게시물 url 컬럼 생성
    df['url'] = df['id'].apply(lambda x : r'https://www.wanted.co.kr/wd/' + str(x))
    #category_tags id 맵핑
    df['category_tags'] = df['category_tags'].apply(lambda x : [category_tags_dict[i] for i in x])


    return df.reset_index(drop=True)


In [33]:
#크롤링 실행 구문


start = time.time()

#조건직무에 맞는 id 크롤링
tag_type_list = [1024, 1025, 10231, 1634, 655] # 데이터사이언티스트, 빅데이터엔지니어, DBA, 머신러닝엔지니어, 데이터엔지니어
id_list = return_id_list(tag_type_list)

end = time.time()
print(end-start)

start = time.time()

#id별 게시물에 접근에서 data 크롤링
df = crawl_job(id_list)

end = time.time()
print(end-start)

df.head()

103.27199387550354
904.1057629585266


due_time                                         skill_tags  \
1     None                                [Java, Python, SQL]   
2     None                                          [C#, C++]   
3     None  [Git, Linux, C / C++, C#, Java, Python, C++, U...   
4     None                                 [iOS, Swift, MVVM]   
5     None  [GraphQL, Hadoop, iOS, MySQL, Pytorch, React, ...   

                                        company_tags  \
1  [Salary Top 1%, Rapid Company Growth, Less tha...   
2  [Salary Top 1%, Rapid Company Growth, 51~300 e...   
3  [Rapid Company Growth, Less than 50 employees,...   
4  [Rapid Company Growth, 51~300 employees, Estab...   
5  [Salary Top 6~10%, Stable Investment, Rapid Co...   

                    position      category_tags address.country  \
1         백엔드 개발자 (빅데이터 플랫폼)   [660, 872, 1025]           Korea   
2  AI researcher for biology             [1634]           Korea   
3             컴퓨터비전/머신러닝 연구원  [900, 1024, 1634]           Korea   
4                DB(DBA, DW)   [655, 872, 1025]           Korea   
5   지그재그 데이터 사이언티스트(추천, 개인화)             [1024]           Korea   

                    address.full_location address.geo_location.n_location.lat  \
1                서울 마포구 새창로 7 SNU장학빌딩 16층                           37.542485   
2  경기도 성남시 분당구 판교역로 152, 알파돔타워 12층 카카오브레인                                None   
3                서울특별시 강남구 강남대로132길 25 4층                           37.512706   
4                                 압구정로148                           37.525203   
5          서울특별시 강남구 테헤란로 521(파르나스타워) 27층                           37.509545   

  address.geo_location.n_location.lng address.geo_location.location_type  \
1                          126.950974                            ROOFTOP   
2                                None                        APPROXIMATE   
3                          127.023015                            ROOFTOP   
4                           127.02507                            ROOFTOP   
5                          127.060869                            ROOFTOP   

  address.location address.country_code  \
1            Seoul                   kr   
2         Gyeonggi                   kr   
3            Seoul                   kr   
4            Seoul                   kr   
5            Seoul                   kr   

                                 detail.requirements  \
1  • 프레임워크를 사용하지 않고 자바만을 이용한 멀티스레딩 프로그래밍 경험\n• JD...   
2  필수조건\n\n- 파이썬 프로그래밍 능숙\n- 딥러닝 프레임워크 (PyTorch, ...   
3  - 석사/박사 학위 소유자 선호\n- 컴퓨터비전 프로젝트 유경험자\n- 전문연구요원...   
4  - SQL Server DBA 운영 업무 경력 2년/7년 이상 또는 그에 준하는 역...   
5  • 주요 업무의 과제 중 한 가지는 자신 있으신 분\n• 데이터 모델링을 통해 비즈...   

                                   detail.main_tasks  \
1  • 로그프레소 기반의 빅데이터 플랫폼 백엔드 서버 개발\n• 로그프레소 백엔드와 프...   
2  - 본 공고를 통해 영입된 분은 아래 프로젝트들에 참여하여 주도적으로 연구 업무를 ...   
3  - 딥러닝 모델 개발 및 경량화\n- 신체 부위 검출기 개발 (얼굴, 눈, 손, 핸...   
4  - AWS Aurora, MySQL DB 운영 및 기술지원\n- DB 장애 대응 및...   
5  • 데이터 모델링을 통한 전사 과제 해결\n• 추천/개인화 모델 고도화\n• 쇼핑몰...   

                             detail.preferred_points company.industry_name  \
1  • 1개 이상의 범용 프로그래밍 언어 경험 (스크립트 언어 포함)\n• 컴퓨터공학에...             IT, Media   
2  우대 조건 (아래 요건 중 최소 3가지 이상의 조건을 보유하신분을 선호합니다)\n\...             IT, Media   
3  - 컴퓨터비전 프로젝트 리딩 경험\n- 신체 부위 검출 및 인식기 개발 경험\n- ...             IT, Media   
4  - 이기종 데이터베이스 운영 및 마이그레이션 경험이 있으신 분\n- 대용량 트래픽 ...             IT, Media   
5  • 프로덕션 레벨의 ML모델 개발 프로세스에 대한 이해가 높고 경험이 있으신 분\n...             IT, Media   

   company.application_response_stats.avg_rate  \
1                                         82.8   
2                                         93.5   
3                                         85.7   
4                                         93.5   
5                                         88.9   

  company.application_response_stats.avg_day address.geo_location  
1                                          3                  NaN  
2                                          4                  NaN  
3                   

In [86]:
df_engineering = engineering(df)

In [88]:
df_engineering.shape

(933, 21)

In [89]:
df_engineering.head()

id due_time                                         skill_tags  \
0   36864     None                                [Java, Python, SQL]   
1  104449     None                                          [C#, C++]   
2    8195     None  [Git, Linux, C / C++, C#, Java, Python, C++, U...   
3   96259     None                                 [iOS, Swift, MVVM]   
4   92165     None  [GraphQL, Hadoop, iOS, MySQL, Pytorch, React, ...   

                                        company_tags  \
0  [Salary Top 1%, Rapid Company Growth, Less tha...   
1  [Salary Top 1%, Rapid Company Growth, 51~300 e...   
2  [Rapid Company Growth, Less than 50 employees,...   
3  [Rapid Company Growth, 51~300 employees, Estab...   
4  [Salary Top 6~10%, Stable Investment, Rapid Co...   

                    position                       category_tags  \
0         백엔드 개발자 (빅데이터 플랫폼)         [자바 개발자, 서버 개발자, 빅데이터 엔지니어]   
1  AI researcher for biology                         [머신러닝 엔지니어]   
2             컴퓨터비전/머신러닝 연구원  [C,C++ 개발자, 데이터 사이언티스트, 머신러닝 엔지니어]   
3                DB(DBA, DW)       [데이터 엔지니어, 서버 개발자, 빅데이터 엔지니어]   
4   지그재그 데이터 사이언티스트(추천, 개인화)                        [데이터 사이언티스트]   

  address.country                   address.full_location  \
0           Korea                서울 마포구 새창로 7 SNU장학빌딩 16층   
1           Korea  경기도 성남시 분당구 판교역로 152, 알파돔타워 12층 카카오브레인   
2           Korea                서울특별시 강남구 강남대로132길 25 4층   
3           Korea                                 압구정로148   
4           Korea          서울특별시 강남구 테헤란로 521(파르나스타워) 27층   

  address.geo_location.n_location.lat address.geo_location.n_location.lng  \
0                           37.542485                          126.950974   
1                                None                                None   
2                           37.512706                          127.023015   
3                           37.525203                           127.02507   
4                           37.509545                          127.060869   

   ... address.location address.country_code  \
0  ...            Seoul                   kr   
1  ...         Gyeonggi                   kr   
2  ...            Seoul                   kr   
3  ...            Seoul                   kr   
4  ...            Seoul                   kr   

                                 detail.requirements  \
0  • 프레임워크를 사용하지 않고 자바만을 이용한 멀티스레딩 프로그래밍 경험\n• JD...   
1  필수조건\n\n- 파이썬 프로그래밍 능숙\n- 딥러닝 프레임워크 (PyTorch, ...   
2  - 석사/박사 학위 소유자 선호\n- 컴퓨터비전 프로젝트 유경험자\n- 전문연구요원...   
3  - SQL Server DBA 운영 업무 경력 2년/7년 이상 또는 그에 준하는 역...   
4  • 주요 업무의 과제 중 한 가지는 자신 있으신 분\n• 데이터 모델링을 통해 비즈...   

                                   detail.main_tasks  \
0  • 로그프레소 기반의 빅데이터 플랫폼 백엔드 서버 개발\n• 로그프레소 백엔드와 프...   
1  - 본 공고를 통해 영입된 분은 아래 프로젝트들에 참여하여 주도적으로 연구 업무를 ...   
2  - 딥러닝 모델 개발 및 경량화\n- 신체 부위 검출기 개발 (얼굴, 눈, 손, 핸...   
3  - AWS Aurora, MySQL DB 운영 및 기술지원\n- DB 장애 대응 및...   
4  • 데이터 모델링을 통한 전사 과제 해결\n• 추천/개인화 모델 고도화\n• 쇼핑몰...   

                             detail.preferred_points company.industry_name  \
0  • 1개 이상의 범용 프로그래밍 언어 경험 (스크립트 언어 포함)\n• 컴퓨터공학에...             IT, Media   
1  우대 조건 (아래 요건 중 최소 3가지 이상의 조건을 보유하신분을 선호합니다)\n\...             IT, Media   
2  - 컴퓨터비전 프로젝트 리딩 경험\n- 신체 부위 검출 및 인식기 개발 경험\n- ...             IT, Media   
3  - 이기종 데이터베이스 운영 및 마이그레이션 경험이 있으신 분\n- 대용량 트래픽 ...             IT, Media   
4  • 프로덕션 레벨의 ML모델 개발 프로세스에 대한 이해가 높고 경험이 있으신 분\n...             IT, Media   

  company.application_response_stats.avg_rate  \
0                                        82.8   
1                                        93.5   
2                                        85.7   
3                                        93.5   
4                                        88.9   

   company.application_response_stats.avg_day         company.name  \
0                                           3                로그프레소   
1                                           4               카카오브레인   
2                                           8         브